In [2]:
%load_ext autoreload
%autoreload
%cd /home/jupyter/MathVGerrmandering_CMAT_2021/
from src import *
from src.nodes import *
from src.mcmc import *

def get_stats(district_type):
    nodes_opts = {
        'abbr'             : 'TX',
        'level'            : 'cntyvtd',
        'district_type'    : district_type,
        'contract_thresh'  : 10,
    }
    N = Nodes(**nodes_opts)
    M = MCMC(random_seed=9000000, nodes_tbl=N.tbl)
    M.post_process1()
    M.post_process2()
    query = f"select * from {M.hash_tbl}"
    df = run_query(query)
    if district_type == 'cd':
        pop_dev = 1
        base_defect = 22
    elif district_type == 'sldu':
        pop_dev = 13
        base_defect = 13
    elif district_type == 'sldl':
        pop_dev = 19
        base_defect = 19
    loose_mask =  df['pop_deviation_plan'] <= pop_dev
    tight_mask = (df['pop_deviation_plan'] <= pop_dev) & (df['defect_plan'] <= base_defect)
    print(f'{district_type} loose mask stats')
    display(df.loc[loose_mask, 'defect_plan'].describe())
    print('{district_type} tight mask stats')
    display(df.loc[tight_mask, 'defect_plan'].describe())
    return df

df_cd   = get_stats('cd')
df_sldl = get_stats('sldu')
df_sldu = get_stats('sldl')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/jupyter/MathVGerrmandering_CMAT_2021
Get crosswalks_TX_2010_tabblock   ... tabblock table exists ... success
Get assignments_TX_2020_tabblock  ... tabblock table exists ... success
Get shapes_TX_2020_tabblock       ... tabblock table exists ... success
Get census_TX_2020_tabblock       ... tabblock table exists ... success
Get elections_TX_2020_tabblock    ... tabblock table exists ... success
Get nodes_TX_2020_cntyvtd_cd_contract10 ... cntyvtd table exists ... success
Stacking summaries & uniquifying ... {district_type} loose mask stats


count    60173.000000
mean        31.348495
std          5.803884
min         19.000000
25%         24.000000
50%         33.000000
75%         36.000000
max         46.000000
Name: defect_plan, dtype: float64

{district_type} tight mask stats


count    7925.000000
mean       21.879621
std         0.375142
min        19.000000
25%        22.000000
50%        22.000000
75%        22.000000
max        22.000000
Name: defect_plan, dtype: float64

Get crosswalks_TX_2010_tabblock   ... tabblock table exists ... success
Get assignments_TX_2020_tabblock  ... tabblock table exists ... success
Get shapes_TX_2020_tabblock       ... tabblock table exists ... success
Get census_TX_2020_tabblock       ... tabblock table exists ... success
Get elections_TX_2020_tabblock    ... tabblock table exists ... success
Get nodes_TX_2020_cntyvtd_sldl_contract10 ... cntyvtd table exists ... success
Stacking summaries & uniquifying ... {district_type} loose mask stats


count    617978.000000
mean         44.933808
std          12.213727
min          16.000000
25%          38.000000
50%          47.000000
75%          57.000000
max          57.000000
Name: defect_plan, dtype: float64

{district_type} tight mask stats


count    66203.000000
mean        18.951906
std          0.268787
min         16.000000
25%         19.000000
50%         19.000000
75%         19.000000
max         19.000000
Name: defect_plan, dtype: float64

Get crosswalks_TX_2010_tabblock   ... tabblock table exists ... success
Get assignments_TX_2020_tabblock  ... tabblock table exists ... success
Get shapes_TX_2020_tabblock       ... tabblock table exists ... success
Get census_TX_2020_tabblock       ... tabblock table exists ... success
Get elections_TX_2020_tabblock    ... tabblock table exists ... success
Get nodes_TX_2020_cntyvtd_sldu_contract10 ... cntyvtd table exists ... success
Stacking summaries & uniquifying ... 

KeyError: 'summaries'

In [26]:
f = f'nodes_TX_2020_tabblock_sldu_contract0'
nodes_tbl = f'{proj_id}.redistricting_data.{f}'
pq = root_path / f'redistricting_data/nodes/TX/{f}.parquet'
pq.parent.mkdir(parents=True, exist_ok=True)
q = f"select * from {nodes_tbl}"
df = run_query(q)
df.to_parquet(pq)
to_gcs(pq)

In [27]:
f = f'nodes_TX_2020_tabblock_sldl_contract0'
nodes_tbl = f'{proj_id}.redistricting_data.{f}'
pq = root_path / f'redistricting_data/nodes/TX/{f}.parquet'
pq.parent.mkdir(parents=True, exist_ok=True)
q = f"select * from {nodes_tbl}"
df = run_query(q)
df.to_parquet(pq)
to_gcs(pq)